# __Neural Networks__
## Exam Project: Adversarial Training for Free!

**Students**:
- **Name**: *Gianmarco Scarano* | Matricola Code: *2047315*<br>
- **Name**: *Giancarlo Tedesco* | Matricola Code: *2057231*

### Google Colab compatibility
Run this next cell only if you're using Google Colab!

In [ ]:
# Library needed for this notebook
!pip install -U tqdm --quiet

# Imports

In [1]:
# We set the global variable for Google Colab, so we can assign possible paths, etc. accordingly
try:
  import google.colab
  RunningInCOLAB = True
except:
  RunningInCOLAB = False

In [2]:
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

print('Dependecies loaded')
print("===================================================")

Dependecies loaded


## Check if we have CUDA support

In [3]:
if(torch.cuda.is_available()):
    device = torch.device("cuda")
    print('Cuda available: {}'.format(torch.cuda.is_available()))
    print("GPU: " + torch.cuda.get_device_name(torch.cuda.current_device()))
    print("Total memory: {:.1f} GB".format((float(torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))))
    print("===================================================")
else:
    device = torch.device("cpu")
    print('Cuda not available, so using CPU. Please consider switching to a GPU runtime before running the notebook!')

Cuda available: True
GPU: NVIDIA GeForce RTX 3060 Laptop GPU
Total memory: 6.0 GB


# Fashion MNIST Dataset

In [4]:
# Data augmentation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [5]:
# Download the train dataset
train_dataset = datasets.FashionMNIST(root ='./', train=True, download=True, transform=transform)

# Download the test dataset
test_dataset = datasets.FashionMNIST(root ='./', train=False, download=True, transform=transform)

# Create train and test data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Model definition: DummyNet

In [6]:
class DummyNet(nn.Module):
    def __init__(self, num_classes=10):
        super(DummyNet, self).__init__()

        # First convolution
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Second convolution
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Third convolution
        self.conv3 = nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Activation functions
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

        # Flatten layer
        self.flatten = nn.Flatten()

        # Fully connected layers
        self.fc1 = nn.Linear(288, 150) # 288 is the output shape of the Flatten() function applied after the 3rd convolutional block.
        self.fc2 = nn.Linear(150, 50)
        self.fc3 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu(self.bn1(self.conv1(x)))) # First convolution
        x = self.pool2(self.relu(self.bn2(self.conv2(x)))) # Second convolution
        x = self.pool3(self.relu(self.bn3(self.conv3(x)))) # Third convolution

        x = self.flatten(x) # Flatten of the output of the 3rd convolutional block

        x = self.relu(self.fc1(x))  # FC1
        x = self.relu(self.fc2(x))  # FC2

        x = self.fc3(x)             # Output layer
        return self.softmax(x)      # Final activation function (SoftMax)

# Training phase

In [7]:
# Let's define some variables for the training phase
lr = 1e-3
momentum = 0.9
epochs = 100

# Our model
model = DummyNet(num_classes=10)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
#optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [ ]:
correct = 0
total = 0

# Train the model
for epoch in range(epochs):

    model.train()
    
    for i, (images, labels) in enumerate(tqdm(train_loader, desc=F"Epoch n.{epoch+1} (Train)")):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        accuracy = correct / len(labels)

    # Test the model
    model.eval()

    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in tqdm(test_loader, desc=F"Epoch n.{epoch+1} (Validation)"):
            outputs = model(images)
            lossVal = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Print the loss and accuracy every 10 epochs
    print(f'Epoch {epoch+1:03}:')     
    print(f'\t- Training accuracy   : {accuracy:.4f}')
    print(f'\t- Training loss       : {loss.item():.4f}')
    print(f'\t- Validation accuracy : {correct / total:.4f}')
    print(f'\t- Validation loss : {lossVal.item():.4f}')
    print("=========================================")